In [1]:
import time
from collections import deque

class sokuban:

    allignment = {
    'down':  { 'move': 'd', 'push': 'D', 'posx':  0, 'posy':  1 },
    'left':  { 'move': 'l', 'push': 'L', 'posx': -1, 'posy':  0 },
    'up':    { 'move': 'u', 'push': 'U', 'posx':  0, 'posy': -1 },
    'right': { 'move': 'r', 'push': 'R', 'posx':  1, 'posy':  0 }
      }

    def __init__(self, level):
        self.level = []
        for r in level:
            self.level.append(r)
    
    def loadLevel(self):
        level = []
        for r in self.level:
            level.append(r)
        return level

    def check_corner(self, p, o):
        testPos = {
          'down':  [[[0, 1],[1, 0], [1, 1]],
                    [[0, 1],[-1, 0], [-1, 1]]],
          'left':  [[[-1, 0],[0, 1], [-1, 1] ],
                    [[-1, 0],[0, -1], [-1, -1]]],
          'up':    [[[0, -1],[1, 0], [1, -1] ],
                    [[0, -1],[-1, 0], [-1, -1]]],
          'right': [[[1, 0],[0, 1], [1, 1]],
                    [[1, 0],[0, -1], [1, -1]]]
        }
        check_corner = False
        for n in [0, 1]:
            tp = testPos[o][n]
            check_corner = check_corner or (self.level[p['y']+tp[0][1]][p['x']+tp[0][0]] in '#$*' and self.level[p['y']+tp[1][1]][p['x']+tp[1][0]] in '#$*' and self.level[p['y']+tp[2][1]][p['x']+tp[2][0]] in '#$*')
        return check_corner

    def expand(self, o):
        (x, y) = self.getsokuban()
        target = ( x + self.allignment[o]['posx'], y + self.allignment[o]['posy'])
        neighbour = self.level[ target[1]][ target[0]]
        return neighbour == levels['symbol']['floor'] or neighbour == levels['symbol']['storage']

    def canPush(self, o):
        result = False
        (x, y) = self.getsokuban()
        target = { 'sokuban': (x + self.allignment[o]['posx'], y + self.allignment[o]['posy'] ),
            'box' : (x + 2*self.allignment[o]['posx'], y + 2*self.allignment[o]['posy'] ) }
        neighbour = self.level[ target['sokuban'][1] ][ target['sokuban'][0] ]
        if neighbour == levels['symbol']['box'] or neighbour == levels['symbol']['boxOnStorage']:
            backneighbour = self.level[ target['box'][1] ][ target['box'][0] ]
            if backneighbour == levels['symbol']['floor'] or backneighbour == levels['symbol']['storage'] :
                result = not ( self.check_corner({ 'y': target['box'][1], 'x': target['box'][0]}, o ) and backneighbour == levels['symbol']['floor'] )
        return result
    
    def move(self, o):
        (x, y) = self.getsokuban()
        target = (x + self.allignment[o]['posx'], y + self.allignment[o]['posy'] )
        self.level[y] = self.level[y][:x] + (levels['symbol']['floor'] if self.level[y][x] == levels['symbol']['sokuban'] else levels['symbol']['storage']) + self.level[ y ][x+1:]
        (x, y) = target
        self.level[y] = self.level[y][:x] + (levels['symbol']['sokuban'] if self.level[y][x] == levels['symbol']['floor'] else levels['symbol']['sokubanOnStorage']) + self.level[y][x+1:]

    def push(self, o):
        (x, y) = self.getsokuban()
        target = { 'sokuban': ( x + self.allignment[o]['posx'], y + self.allignment[o]['posy'] ),
            'box' : (x + 2*self.allignment[o]['posx'], y + 2*self.allignment[o]['posy'] ) }
        self.level[y] = self.level[ y ][:x] + (levels['symbol']['floor'] if self.level[y][x] == levels['symbol']['sokuban'] else levels['symbol']['storage']) + self.level[y][x+1:]
        (x, y) = target['sokuban']
        self.level[ y ] = self.level[y][:x] + (levels['symbol']['sokuban'] if self.level[y][x] == levels['symbol']['box'] else levels['symbol']['sokubanOnStorage']) + self.level[y][x+1:]
        (x, y) = target['box']
        self.level[y] = self.level[y][:x] + (levels['symbol']['box'] if self.level[y][x] == levels['symbol']['floor'] else levels['symbol']['boxOnStorage']) + self.level[y][x+1:]
    
    def isSolved(self):
        result = True
        for r in self.level:
            for c in r:
                if c == levels['symbol']['storage'] or c == levels['symbol']['box'] or  c == levels['symbol']['sokubanOnStorage']:
                    result = False
                    break
            if not result:
                break
        return result

    def getsokuban(self):
        result = None
        for y,r in enumerate(self.level):
            for x,c in enumerate(r):
                if c == levels['symbol']['sokuban'] or c == levels['symbol']['sokubanOnStorage']:
                    result = (x, y)
                    break
            if result:
                break
        return result

    def solve(self):
        path = ''
        queue = deque([[self.loadLevel(), path]])
        visited = set([hash(''.join(self.loadLevel()))])
        while queue:
            level, path = queue.popleft()
            for o in self.allignment:
                s = sokuban(level)
                if s.canPush(o):
                    s.push(o)
                    levelHash = hash(''.join(s.loadLevel()))
                    if levelHash not in visited:
                        if s.isSolved():
                            return path + self.allignment[o]['push']
                    queue.append([s.loadLevel(), path + self.allignment[o]['push']])
                    visited.add(levelHash)
                elif s.expand(o):
                    s.move(o)
                    levelHash = hash(''.join(s.loadLevel()))
                    if levelHash not in visited:
                        queue.append([s.loadLevel(), path + self.allignment[o]['move']])
                        visited.add(levelHash)
        return None


In [2]:
def main():
    level = levels['setup'][0]
    st = time.time()
    s = sokuban(level['board'])
    print('\n'.join(s.level))
    print(level['Level No'])
    print(s.solve())
    end = time.time()
    print(end - st, 'seconds')

In [7]:
# from sokuban import levelno
levelno = int(input("Enter level no: "))
d={}
levels = {
'symbol' : {
    'floor':            " ",
    'wall':             "#",
    'box':              "$",
    'sokuban':          "@",
    'storage':          ".",
    'boxOnStorage':     "*",
    'sokubanOnStorage': "+",
},
'setup' : []
}
val = []
f = open(r'sokubanlevels/level{}.txt'.format(levelno),"r")
lists = f.readlines()
# print(lists)
key = 'board'
key2 = 'Level No'
for i in range(len(lists)):
    val.append(lists[i].strip('\n'))
d[key] = val
d[key2] = "Level "+ str(levelno)
L=levels["setup"]
L.append(d)
main()

Enter level no: 1
#########
#...  . #
######  #
#     $#
# $.   #
# # ####
#@$$ $ #
#      #
####### 
Level 1
drrUUUdddlluuuRuRRdrrUUruLLLLLrrrrdddlluRdrUUruLLLLrrrddlllldRdddrrruLLdlUUUluRRRdrUUruLLLrrddllldllddRdrUUUluRRRdLrrUU
178.33729004859924 seconds


In [8]:
# from sokuban import levelno
levelno = int(input("Enter level no: "))
d={}
levels = {
'symbol' : {
    'floor':            " ",
    'wall':             "#",
    'box':              "$",
    'sokuban':          "@",
    'storage':          ".",
    'boxOnStorage':     "*",
    'sokubanOnStorage': "+",
},
'setup' : []
}
val = []
f = open(r'sokubanlevels/level{}.txt'.format(levelno),"r")
lists = f.readlines()
# print(lists)
key = 'board'
key2 = 'Level No'
for i in range(len(lists)):
    val.append(lists[i].strip('\n'))
d[key] = val
d[key2] = "Level "+ str(levelno)
L=levels["setup"]
L.append(d)
main()

Enter level no: 2
########
#@  #  #
# ..   ##
##.##   #
 #  $$# #
 ###$   #
   #  ###
   ####
Level 2
drrrrdrrddllUUruLLLulldRddrRlluuRRRdrrddlldlUrrruulluurDldDLddrUUUddrruuLuLDDuuLLulldRddRRdRUUruulDlLulDrrrDDlddrUUUrrddLruuluLLLrrddlddrUUUruLL
0.54103684425354 seconds


In [9]:
# from sokuban import levelno
levelno = int(input("Enter level no: "))
d={}
levels = {
'symbol' : {
    'floor':            " ",
    'wall':             "#",
    'box':              "$",
    'sokuban':          "@",
    'storage':          ".",
    'boxOnStorage':     "*",
    'sokubanOnStorage': "+",
},
'setup' : []
}
val = []
f = open(r'sokubanlevels/level{}.txt'.format(levelno),"r")
lists = f.readlines()
# print(lists)
key = 'board'
key2 = 'Level No'
for i in range(len(lists)):
    val.append(lists[i].strip('\n'))
d[key] = val
d[key2] = "Level "+ str(levelno)
L=levels["setup"]
L.append(d)
main()

Enter level no: 3
   ###
  ## # ####
 ##  ###  #
## $      #
#   @$ #  #
### $###  #
  #  #..  #
 ## ##.# ##
 #      ##
 #     ##
 #######
Level 3
lluRRRRRllldlddddrrruurruuruulDlldLLulDDDDurUUrurrrDrddllldddlllluRuuuuuRRRRldllddldddrruruurrruulDrdLLruuruulDDDrdLuuulldllddlddRRdrU
21.046937942504883 seconds


In [10]:
# from sokuban import levelno
levelno = int(input("Enter level no: "))
d={}
levels = {
'symbol' : {
    'floor':            " ",
    'wall':             "#",
    'box':              "$",
    'sokuban':          "@",
    'storage':          ".",
    'boxOnStorage':     "*",
    'sokubanOnStorage': "+",
},
'setup' : []
}
val = []
f = open(r'sokubanlevels/level{}.txt'.format(levelno),"r")
lists = f.readlines()
# print(lists)
key = 'board'
key2 = 'Level No'
for i in range(len(lists)):
    val.append(lists[i].strip('\n'))
d[key] = val
d[key2] = "Level "+ str(levelno)
L=levels["setup"]
L.append(d)
main()

Enter level no: 4
 #########
 #       #
 # ##### #
 # #   # #
 # $.#@# #
###$.  # #
#  $.### #
# #$.    #
#    #####
######
Level 4
dlldddllluurRllddrrrurrrruuuuuulllllldddRDrrruullDldRluluuurrrrrrddddddllllULrddlUUdrrrrruuuuuulllllldddrrDDldRuuuurrddLruulldlDDrUdddllluuRRllddrrUrUddllluurRuuluuurrrrrrddddddllL
3.3741281032562256 seconds


In [11]:
# from sokuban import levelno
levelno = int(input("Enter level no: "))
d={}
levels = {
'symbol' : {
    'floor':            " ",
    'wall':             "#",
    'box':              "$",
    'sokuban':          "@",
    'storage':          ".",
    'boxOnStorage':     "*",
    'sokubanOnStorage': "+",
},
'setup' : []
}
val = []
f = open(r'sokubanlevels/level{}.txt'.format(levelno),"r")
lists = f.readlines()
# print(lists)
key = 'board'
key2 = 'Level No'
for i in range(len(lists)):
    val.append(lists[i].strip('\n'))
d[key] = val
d[key2] = "Level "+ str(levelno)
L=levels["setup"]
L.append(d)
main()

Enter level no: 5
 ########
 #  ##  #
 #.$   .###
 # .## $  #
##$### #  #
# $    # ##
# @ #.    #
#######   #
      #####
Level 5
rurrdrrruuuLulDDDllldlluRRRRdRRdrUllUUUruulDllLdlDururrrDDDllldlluRRRRdRUUUddlllluuruulDDDuurrrRdddllldlluRRRRdrRdrruLLLUlllluuurrrrdrrdDuulluurDlllldlddrrrrUURuLLLrrdddlllluuuurDrrrddddrdrruLuuruLLullllldddrrrruUdddRdrUUUruLLuLLLLulDDDuurrrrDrrdddlluUUddllldlluRuuurrrRdddllLdlUUU
12.902086734771729 seconds


In [12]:
# from sokuban import levelno
levelno = int(input("Enter level no: "))
d={}
levels = {
'symbol' : {
    'floor':            " ",
    'wall':             "#",
    'box':              "$",
    'sokuban':          "@",
    'storage':          ".",
    'boxOnStorage':     "*",
    'sokubanOnStorage': "+",
},
'setup' : []
}
val = []
f = open(r'sokubanlevels/level{}.txt'.format(levelno),"r")
lists = f.readlines()
# print(lists)
key = 'board'
key2 = 'Level No'
for i in range(len(lists)):
    val.append(lists[i].strip('\n'))
d[key] = val
d[key2] = "Level "+ str(levelno)
L=levels["setup"]
L.append(d)
main()

Enter level no: 6
    #####           
    #   #          
    #$  #          
  ###  $##         
  #  $ $ #         
### # ## #   ######
#   # ## #####  ..#
# $  $          ..#
##### ### #@##  ..#
    #     #########
    #######        
Level 6
ullluuuLUllDlldddrRRRRRRRRRRdrUllllllllllllulldRRRRRRRRRRRRRuRRlDllllllluuululldDDuulldddrRRRRRRRRRRdRRlUllllllluuulLulDDDuulldddrRRRRRRRRRRuRDllllllluuulluuurDDluulDDDDDuulldddrRRRRRRRRRRRRlllllllluuuLLulDDDuulldddrRRRRRRRRRRRldR
6567.426945924759 seconds
